In [2]:
# Example from Programming Collective Intelligence, Chapter 6

### Filtering blog feeds ###

In [4]:
from docclass import *
import re
import feedparser   # pip install feedparser

# Takes a filename of URL of a blog feed and classifies the entries
def read(feed, classifier):
    # Get feed entries and loop over them
    f = feedparser.parse(feed)
    for entry in f['entries']:
        print
        print '-----'
        # Print the contents of the entry
        print 'Title:     ' + entry['title'].encode('utf-8')
        print 'Publisher: ' + entry['publisher'].encode('utf-8')
        print
        print entry['summary'].encode('utf-8')
        # Combine all the text to create one item for the classifier
        fulltext = '%s\n%s\n%s' % (entry['title'], entry['publisher'], entry['summary'])
        # Print the best guess at the current category
        print 'Guess: ' + str(classifier.classify(fulltext))
        # Ask the user to specify the correct category and train on that
        cl = raw_input('Enter category: ')
        classifier.train(fulltext, cl.strip())


def entryfeatures(entry):
    splitter = re.compile('\\W*')
    f = {}
    titlewords = [s.lower() for s in splitter.split(entry['title']) if len(s) > 2 and len(s) < 20]
    for w in titlewords: f['Title:' + w] = 1
    summarywords = [s.lower() for s in splitter.split(entry['summary']) if len(s) > 2 and len(s) < 20]
    # Count uppercase words
    uc = 0
    for i in range(len(summarywords)):
        w = summarywords[i]
        f[w] = 1
        if w.isupper(): uc += 1
        # Get word pairs in summary as features
        if i < len(summarywords) - 1:
            twowords = ' '.join(summarywords[i:i + 1])
            f[twowords] = 1

    # Keep creator and publisher whole
    f['Publisher:' + entry['publisher']] = 1
    # UPPERCASE is a virtual word flagging too much shouting
    if float(uc) / len(summarywords) > 0.3: f['UPPERCASE'] = 1
    return f

Train discriminant Fisher classifier:

In [ ]:
cl = fisherclassifier(getwords)
read('python_search.xml',cl)


-----
Title:     My new baby boy!
Publisher: Shetan Noir, the zombie belly dancer! - MySpace Blog

THis is my new baby, Anthem. He is a 3 and half month old ball <b>python</b>, orange shaded normal pattern. I have held him about 5 time since I brought him home tonight at 8:00pm...
Guess: None
Enter category: test
My new baby boy!
Shetan Noir, the zombie belly dancer! - MySpace Blog
THis is my new baby, Anthem. He is a 3 and half month old ball <b>python</b>, orange shaded normal pattern. I have held him about 5 time since I brought him home tonight at 8:00pm...

-----
Title:     If you need a laugh...
Publisher: Kate&#39;s space

Even does 'funny walks' from Monty <b>Python</b>. He talks about all the ol' ladies that are after him. He teases me about my horror obsession. He attempts suicide. And best of all, he talks about poo. Who doesn't think poo is funny???!
If you need a laugh...
Kate&#39;s space
Even does 'funny walks' from Monty <b>Python</b>. He talks about all the ol' ladies 

Calculate probabilities:

In [ ]:
print(cl.cprob('python','prog'))
print(cl.cprob('python','snake'))
print(cl.cprob('python','monty'))
print(cl.cprob('eric','monty'))
print(cl.fprob('eric','monty'))